In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
root_dataset=os.path.join("..")
dataset_dir=os.path.join(root_dataset,"dataset")
dataset_train_path=os.path.join(dataset_dir,"train","Task1_2_180724_reformatted.csv")

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer

In [4]:
torch.range(4,4).to("cuda")

/tmp/ipykernel_8028/2338294619.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  torch.range(4,4).to("cuda")


tensor([4.], device='cuda:0')

In [5]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: A100-SXM-80GB
Compute Capability: (8, 0)
Total Memory: 85199093760 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 3.10.0-1160.el7.x86_64
Python Version: 3.10.12


In [6]:
# Pre trained model
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct" 

# # Dataset name
# dataset_name = "vicgalle/alpaca-gpt4"

# Hugging face repository link to save fine-tuned model(Create new repository in huggingface,copy and paste here)
new_model = "t1"

In [7]:
import json
import os
import glob
from pathlib import Path
law_dir=os.path.join(root_dataset,"wcx_ptt_export_280524")
law_list=glob.glob(os.path.join(law_dir,"*.json"))
law_data = {}
for i in law_list:
    key=Path(i).stem
    law_data[key]={}
    with open(i) as json_file:
        json_data = json.load(json_file)
    for j in json_data:
        law_data[key][j['section_num']]=j['section_content']

In [8]:
from datasets import Dataset,DatasetDict
import pandas as pd
df = pd.read_csv(dataset_train_path)
ds = Dataset.from_pandas(df)
dataset=ds

In [9]:
# Load base model(llama-2-7b-hf) and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
dataset

Dataset({
    features: ['Unnamed: 0', 'idx', 'question', 'question_ori', 'answer', 'answer_ori', 'relevant_laws', 'irrelevant_laws', 'comment_1_1', 'comment_1_2'],
    num_rows: 2392
})

In [11]:
import datetime
dataset.filter(lambda example: isinstance(eval(example['relevant_laws']),list))

Filter:   0%|          | 0/2392 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'idx', 'question', 'question_ori', 'answer', 'answer_ori', 'relevant_laws', 'irrelevant_laws', 'comment_1_1', 'comment_1_2'],
    num_rows: 2392
})

In [12]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["question"]
    outputs      = examples["answer"]
    list_law= [i for i in examples["relevant_laws"]]
    # list_txt_law=[]
    # for i in list_law:
    #     list_txt_law.append(law_data[i["law_code"][str(i['sections'][0])]])
    texts = []
    for i, output,law in zip(inputs, outputs,list_law):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        list_txt_law=[]
        for k in eval(law):
            if k["law_code"] not in set(law_data.keys()):
                pass
            else:
                if str(k['sections'][0]) not in set(law_data[k["law_code"]].keys()):
                    pass
                else:
                    list_txt_law.append(law_data[k["law_code"]][str(k['sections'][0])].strip())
        if list_txt_law!=[]:
            source_information="- "+"\n- ".join(list_txt_law)
            source_information=source_information.strip()
            i=i.strip()
            content=f"""Query: {i}\nContinue to answer the query by using the Search Results:\n{source_information}"""
            text = tokenizer.apply_chat_template([{"role":"user","content":content},{"role":"assistant","content":output.strip()}], tokenize=False) + EOS_TOKEN
            texts.append(text)
        else:
            texts.append(None)
    return { "text" : texts, }

from datasets import load_dataset
dataset = dataset.map(formatting_prompts_func, batched = True,).filter(lambda example: isinstance(example['text'],str))

Map:   0%|          | 0/2392 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2392 [00:00<?, ? examples/s]

In [13]:
dataset

Dataset({
    features: ['Unnamed: 0', 'idx', 'question', 'question_ori', 'answer', 'answer_ori', 'relevant_laws', 'irrelevant_laws', 'comment_1_1', 'comment_1_2', 'text'],
    num_rows: 1533
})

In [14]:
print(dataset["text"][228])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Query: คณะกรรมการใดที่มีอำนาจพิจารณาอุทธรณ์เกี่ยวกับการจัดซื้อจัดจ้างภาครัฐ
Continue to answer the query by using the Search Results:
- พระราชบัญญัติการจัดซื้อจัดจ้างและการบริหารพัสดุภาครัฐ พ.ศ. 2560 มาตรา 43 ให้คณะกรรมการพิจารณาอุทธรณ์มีอำนาจหน้าที่ ดังต่อไปนี้
(1) พิจารณาและวินิจฉัยอุทธรณ์ตามมาตรา 119
(2) พิจารณาข้อร้องเรียนกรณีที่เห็นว่าหน่วยงานของรัฐมิได้ปฏิบัติให้เป็นไปตามแนวทางของพระราชบัญญัตินี้ กฎกระทรวง หรือระเบียบที่ออกตามความในพระราชบัญญัตินี้
(3) จัดทำรายงานเกี่ยวกับปัญหาและอุปสรรคในการดำเนินการพิจารณาอุทธรณ์และข้อร้องเรียนเสนอคณะกรรมการนโยบายอย่างน้อยปีละหนึ่งครั้ง
(4) ปฏิบัติหน้าที่อื่นตามที่กำหนดในพระราชบัญญัตินี้หรือตามที่คณะกรรมการนโยบาย รัฐมนตรี หรือคณะรัฐมนตรีมอบหมาย
ผลการดำเนินการตาม (1) (2) และ (3) ให้ประกาศในระบบเครือข่ายสารสนเทศของกรมบัญชีกลางตามวิธีการที่กรมบัญชีกลางกำหนด
การยื่นข้อร้องเรียนและการพิจารณาข้อร้องเรียนตาม (2) ให้เป็นไปตามระเบียบที่รัฐมนตรีกำหนด
ในกรณีที่พิจารณาข้อร้องเรียนตาม (2) แล้วรับฟัง

In [15]:
peft_config = LoraConfig(
    lora_alpha= 8,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [16]:
training_arguments = TrainingArguments(
    output_dir= "./results_mark2",
    num_train_epochs= 1,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="wandb",
)

In [17]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
) 

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1533 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
# Train model
trainer.train()

[2024-07-30 07:27:26,882] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wannaphong. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
